In [ ]:
+# improt library need from torhc and hugging face
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score

The CustomDataset class encapsulates the functionality required to prepare the dataset for training a sequence classification model using BERT. It handles tasks such as tokenization, padding, and organizing the data into batches for efficient training.

In [ ]:
# Define a custom dataset class  # questo in case sono io create il date sete
# I need this funtion for preparing the data # by the way I can crete more detaail and custome fucntion for the purprose fo my task summatiziason or classification thsi is simple for classificaitonand can be mroe speccifi
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }





In [ ]:
import random

# Define positive and negative sentences
positive_sentences = [
    "I love sunny days.",
    "The cake was delicious.",
    "She is very talented.",
    "This movie is amazing.",
    "I enjoy spending time with friends.",
    "The flowers in the garden are beautiful.",
    "He always makes me laugh.",
    "I'm grateful for the kindness of strangers.",
    "The sunset was breathtaking.",
    "I feel lucky to have such supportive friends."
]


negative_sentences = [
    "I hate rainy weather.",
    "The food tasted awful.",
    "He is always late.",
    "The service was terrible.",
    "I'm tired of this endless rain.",
    "The traffic was unbearable.",
    "I'm frustrated with the lack of progress.",
    "This headache won't go away.",
    "I'm disappointed with the quality of the product.",
    "The news is always so depressing."
]


# defines
texts = positive_sentences[:10] + negative_sentences[:10]
labels = [1] * 10 + [0] * 10

print(texts)
print(labels)

['I love sunny days.', 'The cake was delicious.', 'She is very talented.', 'This movie is amazing.', 'I enjoy spending time with friends.', 'The flowers in the garden are beautiful.', 'He always makes me laugh.', "I'm grateful for the kindness of strangers.", 'The sunset was breathtaking.', 'I feel lucky to have such supportive friends.', 'I hate rainy weather.', 'The food tasted awful.', 'He is always late.', 'The service was terrible.', "I'm tired of this endless rain.", 'The traffic was unbearable.', "I'm frustrated with the lack of progress.", "This headache won't go away.", "I'm disappointed with the quality of the product.", 'The news is always so depressing.']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:


# Initialize the BERT tokenizer and model, load the pre traiend tokenizion
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# this line load the pre trained weights for the the bart sequnce model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# I have created this function becasue to test the model before and after fien tunning

def test_model(model, tokenizer, texts, labels):
    """
    Test the model on given texts and labels.

    Args:
    - model: Pre-trained BERT model.
    - tokenizer: Pre-trained BERT tokenizer.
    - texts: List of text samples.
    - labels: List of corresponding labels.

    Returns:
    - accuracy: Accuracy of the model on the given dataset.
    """
    # Tokenize input texts  #i tokeinier coems form the tokenziaito pre triane token tahat I have imoei fo that model v
    encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')




    # By wrapping the forward pass inside torch.no_grad(), we ensure that no gradients are calculated during this operation,
    # which is desirable during inference or testing to save computational resources and avoid unintended modifications to
    # model parameters.



    # input_ids contains token IDs for the input text, attention_mask indicates which tokens are real and which are padding, and token_type_ids indicates the segment to which each token belongs (if applicable).
    #During the forward pass, the model utilizes this information to process the input text and generate predictions.

    # Forward pass
    with torch.no_grad():
        outputs = model(**encoded_texts)

    # Get predictions
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    return accuracy



test_texts =  [ "I love sunny days.",  "I hate sunny days."]
test_labels = [1 ,0]  # Example labels (1 for positive, 0 for negative)


# Test the model before fine-tuning
before_fine_tuning_accuracy = test_model(model, tokenizer, test_texts, test_labels)
print(f"Accuracy before fine-tuning: {before_fine_tuning_accuracy:.2f}")



Accuracy before fine-tuning: 1.00


In [ ]:

# Define the dataset and dataloader # this is in case I ma usign my kind of datset there is an alterantive which is data_loader from dataste library
dataset = CustomDataset(texts, labels, tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


print(dataset[1])

{'input_ids': tensor([  101,  1996,  9850,  2001, 12090,  1012,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

<ipython-input-2-11e203b4bc95>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(label, dtype=torch.long)


In [ ]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Fine-tune the model
model.train()
for epoch in range(5):  # Adjust number of epochs as needed
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_bert_model")

Epoch 1, Loss: 0.8016283631324768
Epoch 2, Loss: 0.6290516704320908
Epoch 3, Loss: 0.3962341070175171
Epoch 4, Loss: 0.14034517109394073
Epoch 5, Loss: 0.037743990682065486


In [ ]:
after_fine_tuning_accuracy = test_model(model, tokenizer, test_texts, test_labels)
print(f"Accuracy after fine-tuning: {after_fine_tuning_accuracy:.2f}")

Accuracy after fine-tuning: 0.50
